In [37]:
import pandas as pd
import numpy as np

In [ ]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [ ]:
akas = akas.replace({'\\N':np.nan})
akas = akas[akas['region'].str.contains('US',case=False,na=False)]

In [ ]:
basics = basics.replace({'\\N':np.nan})
basics = basics.dropna(axis=0, subset=['runtimeMinutes'])
basics = basics.dropna(axis=0, subset=['genres'])
basics = basics[basics['titleType'].str.contains('Movie',case=False,na=False)]
basics = basics.drop(basics[basics['startYear'] < '2000'].index)
basics = basics.drop(basics[basics['startYear'] > '2022'].index)
basics = basics[~basics['genres'].str.contains('documentary',case=False,na=False)]
basics = basics[basics['tconst'].isin(akas['titleId'])]

In [ ]:
ratings = ratings.replace({'\\N':np.nan})
ratings = ratings[ratings['tconst'].isin(akas['titleId'])]

In [ ]:
print(basics.info())
print(akas.info())
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
Index: 99394 entries, 34800 to 10248547
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          99394 non-null  object
 1   titleType       99394 non-null  object
 2   primaryTitle    99393 non-null  object
 3   originalTitle   99393 non-null  object
 4   isAdult         99394 non-null  object
 5   startYear       96585 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  99394 non-null  object
 8   genres          99394 non-null  object
dtypes: object(9)
memory usage: 7.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 1472351 entries, 5 to 37526954
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1472351 non-null  object
 1   ordering         1472351 non-null  int64 
 2   title            1472349 non-null  object
 3   region       

In [ ]:
## Save current dataframe to file.
basics.to_csv("title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("title_ratings.csv.gz",compression='gzip',index=False)


Your Stakeholder Wants More Data!

In [40]:
import os, time,json
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [42]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [43]:
def get_movie_with_rating(movie_id):
   # Get the movie object for the current id
   movie = tmdb.Movies(movie_id)
   # save the .info .releases dictionaries
   info = movie.info()
   releases = movie.releases()
   # Loop through countries in releases
   for c in releases['countries']:
      # if the country abbreviation==US
      if c['iso_3166_1' ] =='US':
         ## save a "certification" key in the info dict with the certification
         info['certification'] = c['certification']
   return info

In [44]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv(r'C:\Users\CHBAM\OneDrive - MMI HOLDINGS LTD\Documents\GitHub\DATA-ENRICHMENT\Data\title_basics.csv.gz')

In [45]:
YEARS_TO_GET = [2000,2001]
errors = [ ]

In [ ]:
import json
with open('/Users/CHBAM/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key', 'API Read Access Token'])

In [ ]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [ ]:
test_Avengers = get_movie_with_rating("tt0848228") 
test_Avengers

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 118.426,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [ ]:
test_Notebook = get_movie_with_rating("tt0332280") 
test_Notebook

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 74.884,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/mevhneWSqbjU22D1MXNd4H9x0r0.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [46]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
   #Defining the JSON file to store results for year
   JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
   # Check if file exists
   file_exists = os.path.isfile(JSON_FILE)
   # If it does not exist: create it
   if file_exists == False:
   # save an empty dict with just "imdb_id" to the new json file.
      with open(JSON_FILE,'w') as f:
         json.dump([{'imdb_id':0}],f)
   #Saving new year as the current df
   df = basics.loc[ basics['startYear']==YEAR].copy()
   # saving movie ids to list
   movie_ids = df['tconst'].copy()
   # Load existing data from json into a dataframe called "previous_df"
   previous_df = pd.read_json(JSON_FILE)
   # filter out any ids that are already in the JSON_FILE
   movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
   #Get index and movie id from list
   # INNER Loop
   for movie_id in tqdm_notebook(movie_ids_to_get,
                                 desc=f'Movies from {YEAR}',
                                 position=1,
                                 leave=True):
      try:
         # Retrieve then data for the movie id
         temp = get_movie_with_rating(movie_id)  
         # Append/extend results to existing file using a pre-made function
         write_json(temp,JSON_FILE)
         # Short 20 ms sleep to prevent overwhelming server
         time.sleep(0.02)
      except Exception as e:
         errors.append([movie_id, e])

   final_year_df = pd.read_json(JSON_FILE)
   final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
   print(f"- Total errors: {len(errors)}")
   


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1769 [00:00<?, ?it/s]

- Total errors: 270


Movies from 2001:   0%|          | 0/1858 [00:00<?, ?it/s]

- Total errors: 551
